In [2]:
# %load data.py
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import os
import glob
import skimage.io as io
import skimage.transform as trans

Using TensorFlow backend.


In [ ]:
Sky = [128,128,128]
Building = [128,0,0]
Pole = [192,192,128]
Road = [128,64,128]
Pavement = [60,40,222]
Tree = [128,128,0]
SignSymbol = [192,128,128]
Fence = [64,64,128]
Car = [64,0,128]
Pedestrian = [64,64,0]
Bicyclist = [0,128,192]
Unlabelled = [0,0,0]

COLOR_DICT = np.array([Sky, Building, Pole, Road, Pavement,
                          Tree, SignSymbol, Fence, Car, Pedestrian, Bicyclist, Unlabelled])

# def adjustData : 이미지 인덱스에 해당하는 마스크를 해당 픽셀로 씌움.
# adjustData 객체 선언(img,mask,flag_multi_class,num_class)
def adjustData(img,mask,flag_multi_class,num_class):
    if(flag_multi_class):
        # RGB값을 이용한 이미지 정규화, 0~1 범위로 바꿔줌.
        img = img / 255
        # len(s) : 입력값 s의 길이(요소의 전체 개수)를 리턴
        # shape : 모든 배열은 각 차원의 크기를 알려주는 튜플. 차원의 구조를 볼때 사용 (행,렬)로 표시됨
        # [:,:,:,0] : [1차원 모든 행, 2차원 모든 행, 3차원 모든 행, 4차원 행렬 0]
        # mask.shape가 4면 4차원으로 표기하고 아니면 3차원으로 표기함
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        # np.zeros() : 괄호 안에 쓴 숫자 개수만큼의 모든 값이 0인 배열을 생성
        new_mask = np.zeros(mask.shape + (num_class,))
        # 이미지별로 인덱스 찾아서 인덱스에 해당하는 마스크를 해당 픽셀에 씌우는 부분
        for i in range(num_class):
            new_mask[mask == i,i] = 1
        # reshape : mask의 array의 shape를 재설정
        new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    # np.max() : 리스트안에 최대값을 반환함. 행에 대한 열의 값 개수가 일정한 리스트에서만 한가지 최대값을 반환하고 행마다 열의 수가 다른 리스트에서는 최대값이 들어있는 행을 반환함.
    elif(np.max(img) > 1):
        # RGB값을 이용한 이미지 정규화, 0~1 범위로 바꿔줌.
        img = img / 255
        mask = mask /255
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    # img, mask 값으로 리턴
    return (img,mask)


# def trainGenerator : 훈련시킬 이미지 데이터와 마스크 데이터 부풀리기. 두 데이터를 동시에 생성함.
# trainGenerator 객체 선언 : 선언한 객체의 특성을 image_generator와 mask_generator에 적용함.
def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (256,256),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    # 이미지 데이터 생성
    # flow_from_directory : 폴더 형태로된 데이터 구조 가져옴.
    # train_path : 경로
    # classes : 객체
    # class_mode : 데이터 라벨 타입 설정 (binary, categorical, sparse)
    # color_mode :  컬러 타입 설정 (컬러는 rgb, 흑백은 grayscale, 생략하면 컬러로 처리함)
    # target_size : 이미지의 크기를 지정 (width, height)
    # batch_size : 한번에 처리해야할 데이터의 양 설정
    # save_to_dir : 저장할 위치
    # save_prefix  : 저장할 파일 이름 (예 : 'image', 'mask')
    # seed : 랜덤 seed 고정시키기
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    # 마스크 데이터 생성
    # flow_from_directory : 폴더 형태로된 데이터 구조 가져옴.
    # train_path : 경로
    # classes : 객체
    # class_mode : 데이터 라벨 타입 설정 (binary, categorical, sparse)
    # color_mode :  컬러 타입 설정 (컬러는 rgb, 흑백은 grayscale, 생략하면 컬러로 처리함)
    # target_size : 이미지의 크기를 지정 (width, height)
    # batch_size : 한번에 처리해야할 데이터의 양 설정
    # save_to_dir : 저장할 위치
    # save_prefix  : 저장할 파일 이름 (예 : 'image', 'mask')
    # seed : 랜덤 seed 고정시키기
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    # zip() : 동일한 개수로 이루어진 자료형을 묶어줌
    # train_generator : image_generator와 mask_generator의 범위로 묶음.
    train_generator = zip(image_generator, mask_generator)
        # 앞서 선언한 adjustData를 통해 이미지 인덱스에 해당하는 마스크를 해당 픽셀로 씌움.
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        # img, mask 값으로 산출
        yield (img,mask)


# def testGenerator : test 데이터의 개수 및 pixel size를 설정하고 실제 test 데이터 입력시 resize를 진행함.
def testGenerator(test_path,num_image = 30,target_size = (256,256),flag_multi_class = False,as_gray = True):
    for i in range(num_image):
        # test_path 경로에서 grayscale로 이미지 출력.
        img = io.imread(n(test_path,"%d.png"%i),as_gray = as_gray)
        # RGB값을 이용한 이미지 정규화, 0~1 범위로 바꿔줌.
        img = img / 255
        # img를 targe_size(256,256)에 맞춰서 resize
        img = trans.resize(img,target_size)
        # reshape : img의 array의 shape를 재설정
        img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
        img = np.reshape(img,(1,)+img.shape)
        yield img

# def geneTrainNpy : 리스트를 생성하고 adjustData를 수행하여 행렬(배열)로 만듦. 
def geneTrainNpy(image_path,mask_path,flag_multi_class = False,num_class = 2,image_prefix = "image",mask_prefix = "mask",image_as_gray = True,mask_as_gray = True):
    # glob.glob(path) : 경로에 대응되는 모든 파일 및 디렉터리의 이터레이터를 반환함. 한번에 모든 결과를 리스트에 담지 않으므로, 결과가 매우 많은 경우 유용함.
    image_name_arr = glob.glob(os.path.join(image_path,"%s*.png"%image_prefix))
    # image_arr, mask_arr 리스트 생성
    image_arr = []
    mask_arr = []
    # for in enumerate : 리스트가 있는 경우 순서와 리스트의 값을 전달하는 기능. 순서가 있는 자료형(리스트, 튜플, 문자열)을 입력으로 받아 인덱스 값을 포함하는 enumerate 객체를 리턴함.
    for index,item in enumerate(image_name_arr):
        # image를 grayscale로 출력함
        img = io.imread(item,as_gray = image_as_gray)
        # reshape : img의 array의 shape를 재설정
        img = np.reshape(img,img.shape + (1,)) if image_as_gray else img
        mask = io.imread(item.replace(image_path,mask_path).replace(image_prefix,mask_prefix),as_gray = mask_as_gray)
        # reshape : mask의 array의 shape를 재설정
        mask = np.reshape(mask,mask.shape + (1,)) if mask_as_gray else mask
        # img, mask에 앞서 정의한 adjustData를 수행함.
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        # append() : 리스트 맨 뒤에 adjustData를 수행한 img, mask 요소추가
        image_arr.append(img)
        mask_arr.append(mask)
    # np.array() : image_arr, mask_arr을 행렬(배열)로 만들기
    image_arr = np.array(image_arr)
    mask_arr = np.array(mask_arr)
    # 행렬로 만든 image_arr, mask_arr을 리턴시킴.
    return image_arr,mask_arr

# def labelVisualize : num_clss, color_dict, img 객체를 선언하고 num_class에 맞춰 img에 생성한 color_dict리스트를 씌움.
def labelVisualize(num_class,color_dict,img):
     # len(s) : 입력값 s의 길이(요소의 전체 개수)를 리턴 (3차원 배열인지 확인)
    img = img[:,:,0] if len(img.shape) == 3 else img
    # np.zeros() : 괄호 안에 쓴 숫자 개수만큼의 모든 값이 0인 배열을 생성
    img_out = np.zeros(img.shape + (3,))
    # for in range : 입력받은 숫자에 해당되는 범위의 값을 반복 가능한 객체로 만들어 리턴함.
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    # img_out을 RGB값을 이용하여 0~1범위로 정규화하여 리턴
    return img_out / 255


# def saveResult : save_path에 png 형태로 결과 저장
def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    # for in enumerate : 리스트가 있는 경우 순서와 리스트의 값을 전달하는 기능. 순서가 있는 자료형(리스트, 튜플, 문자열)을 입력으로 받아 인덱스 값을 포함하는 enumerate 객체를 리턴함.
    for i,item in enumerate(npyfile):
        # img에 선언한 labelVisualize를 수행하여 imsave를 이용해 save_path에 png 형태로 결과 저장
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)